# 薬剤ごとのPTlist作成

In [ ]:
#ライブラリインポート
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns

#from sklearn.model_selection import train_test_split
#from sklearn.linear_model import LogisticRegression

#標準化用
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

import pandas as pd
#import scipy.stats as sp
import pickle
import numpy
import joblib

import matplotlib.cm as cm
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

from sklearn.metrics.cluster import adjusted_rand_score
from scipy.cluster.hierarchy import fcluster
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

In [ ]:
c_c_death_PT_ID = numpy.load('all_cancer/data/c_c_death_PT_ID.npy')
c_c_death_PT_ID = list(c_c_death_PT_ID)
c_c_PT_ID = pd.read_csv('all_cancer/data/point_inj/all/list_all_cancer_p_or_inj_without_age.csv')
c_c_PT_ID = list(c_c_PT_ID['0'])
len(c_c_PT_ID)

In [ ]:
#inj読み込み
df_inj_opz = pd.read_pickle('all_cancer/df_inj_cancer.pkl')
#経口薬（処方）
parsed_df_prescrip = pd.read_pickle('all_cancer/parsed_df_prescrip.pkl')

In [ ]:
#parsed_df_prescripきれいにしたい
def preprocess_parsed_df_prescrip(parsed_df_prescrip):
    ##df経口投与薬の処理
    clean_parsed_df_prescrip = parsed_df_prescrip.drop(['入力日', '診療科コード', '診療科', '一日回数', '用法', '薬剤コード', '一回量', '一回量単位', '一日量', '一日量単位', '処方期間(日数)', '指示用量', '指示単位', '用法種別', '用法コメント'], axis = 1)
    clean_parsed_df_prescrip['薬価基準収載医薬品コード'].value_counts()
    #このうち抗がん剤のみ抽出する
    clean_parsed_df_prescrip_423 = clean_parsed_df_prescrip[parsed_df_prescrip['薬価基準収載医薬品コード'].str.contains('423', na = False)]
    clean_parsed_df_prescrip_424 = clean_parsed_df_prescrip[parsed_df_prescrip['薬価基準収載医薬品コード'].str.contains('424', na = False)]
    clean_parsed_df_prescrip_429 = clean_parsed_df_prescrip[parsed_df_prescrip['薬価基準収載医薬品コード'].str.contains('429', na = False)]
    clean_parsed_df_prescrip = pd.concat([clean_parsed_df_prescrip_423, clean_parsed_df_prescrip_424, clean_parsed_df_prescrip_429])
    
    ###########頭文字でないのたくさんまぎれていたので消したい
    list_non_head_exact = ['6429700Q1021', '6429008F1020', '6429006F1039', '6429001R1032', '1190012F4238', '1190012F4246', '1190012F4297', '1339005F1423', '2139005F2423', '2149040F4246', '2149040F4297', '2149041F4232', '2149041F4240', '2171022F2424', '2171022F4230', '2171022F4249', '2171022F4290', '2190021F1429', '2325001F1424', '2325003F4236', '2325003F4244', '2329002D4248', '3999424D1029', '3999424G1025', '3999424G2021', '3999424G3028', '3999424G4024', '3999429G1028', '3999429G2024']
    for j in list_non_head_exact:
            drop_index_pp = clean_parsed_df_prescrip.index[clean_parsed_df_prescrip['薬価基準収載医薬品コード'] == str(j)]
            clean_parsed_df_prescrip = clean_parsed_df_prescrip.drop(drop_index_pp)
    ###########
    clean_parsed_df_prescrip['有＝１'] = 1
    return clean_parsed_df_prescrip

clean_parsed_df_prescrip=preprocess_parsed_df_prescrip(parsed_df_prescrip)
pt_id_pp_cancer = sorted(list(set(clean_parsed_df_prescrip['PT_ID'])))

In [ ]:
#injきれいにしたい
def preprocess_df_inj_opz(df_inj_opz):
    clean_df_inj_opz = df_inj_opz.drop(['入力日', '薬剤コード', '診療科コード', '診療科', '入力時刻', '更新時刻','更新日','キャンセル日','キャンセル時刻','キャンセル種別','指示用量','指示単位','成分単位量','成分単位','手技名','用法内容','実施状況','入外区分'], axis = 1)
    clean_df_inj_opz['薬価基準収載医薬品コード'].value_counts()
    #このうち抗がん剤のみ抽出する
    clean_df_inj_opz_423 = clean_df_inj_opz[df_inj_opz['薬価基準収載医薬品コード'].str.contains('423', na = False)]
    clean_df_inj_opz_424 = clean_df_inj_opz[df_inj_opz['薬価基準収載医薬品コード'].str.contains('424', na = False)]
    clean_df_inj_opz_429 = clean_df_inj_opz[df_inj_opz['薬価基準収載医薬品コード'].str.contains('429', na = False)]
    clean_df_inj_opz = pd.concat([clean_df_inj_opz_423, clean_df_inj_opz_424, clean_df_inj_opz_429])
    
    ###########頭文字でないの9個まぎれていたので消したい
    ##'3999424G1025','6343424D3019','1242405A1038','1242401A1315','6343424D2012','1242406G1035','6363424D3035', '3999423A1125','3999423A1044'
    drop_index_inj3999424G1025 = clean_df_inj_opz.index[clean_df_inj_opz['薬価基準収載医薬品コード'] == '3999424G1025']
    clean_df_inj_opz = clean_df_inj_opz.drop(drop_index_inj3999424G1025)
    drop_index_inj6343424D3019 = clean_df_inj_opz.index[clean_df_inj_opz['薬価基準収載医薬品コード'] == '6343424D3019']
    clean_df_inj_opz = clean_df_inj_opz.drop(drop_index_inj6343424D3019)
    drop_index_inj1242405A1038 = clean_df_inj_opz.index[clean_df_inj_opz['薬価基準収載医薬品コード'] == '1242405A1038']
    clean_df_inj_opz = clean_df_inj_opz.drop(drop_index_inj1242405A1038)
    drop_index_inj1242401A1315 = clean_df_inj_opz.index[clean_df_inj_opz['薬価基準収載医薬品コード'] == '1242401A1315']
    clean_df_inj_opz = clean_df_inj_opz.drop(drop_index_inj1242401A1315)
    drop_index_inj6343424D2012 = clean_df_inj_opz.index[clean_df_inj_opz['薬価基準収載医薬品コード'] == '6343424D2012']
    clean_df_inj_opz = clean_df_inj_opz.drop(drop_index_inj6343424D2012)
    drop_index_inj1242406G1035 = clean_df_inj_opz.index[clean_df_inj_opz['薬価基準収載医薬品コード'] == '1242406G1035']
    clean_df_inj_opz = clean_df_inj_opz.drop(drop_index_inj1242406G1035)
    drop_index_inj6363424D3035 = clean_df_inj_opz.index[clean_df_inj_opz['薬価基準収載医薬品コード'] == '6363424D3035']
    clean_df_inj_opz = clean_df_inj_opz.drop(drop_index_inj6363424D3035)
    drop_index_inj3999423A1125 = clean_df_inj_opz.index[clean_df_inj_opz['薬価基準収載医薬品コード'] == '3999423A1125']
    clean_df_inj_opz = clean_df_inj_opz.drop(drop_index_inj3999423A1125)
    drop_index_inj3999423A1044 = clean_df_inj_opz.index[clean_df_inj_opz['薬価基準収載医薬品コード'] == '3999423A1044']
    clean_df_inj_opz = clean_df_inj_opz.drop(drop_index_inj3999423A1044)
    ###########
    clean_df_inj_opz['有＝１'] = 1  
    return clean_df_inj_opz

clean_df_inj_opz=preprocess_df_inj_opz(df_inj_opz)
pt_id_inj_cancer= sorted(list(set(clean_df_inj_opz['PT_ID'])))

In [ ]:
import collections

#prescriptionで抗がん剤出現回数
list_clean_parsed_df_prescrip = list(clean_parsed_df_prescrip['薬価基準収載医薬品コード'])
c = collections.Counter(list_clean_parsed_df_prescrip)
df_most_common_prescrip = pd.DataFrame(c.most_common(), columns = ['薬価基準収載医薬品コード', 'counts'])
df_most_common_prescrip

#薬価基準収載医薬品コードと薬剤名の対応表を作成する
clean_parsed_df_prescrip_for_correspondence = clean_parsed_df_prescrip.drop(['PT_ID', 'Date', '有＝１'], axis =1)
clean_parsed_df_prescrip_for_correspondence = clean_parsed_df_prescrip_for_correspondence.drop_duplicates(subset = '薬価基準収載医薬品コード')
clean_parsed_df_prescrip_for_correspondence

clean_parsed_df_prescrip_for_correspondence = pd.merge(df_most_common_prescrip, clean_parsed_df_prescrip_for_correspondence, on = '薬価基準収載医薬品コード')

#injectionで抗がん剤出現回数
list_clean_df_inj_opz = list(clean_df_inj_opz['薬価基準収載医薬品コード'])
c = collections.Counter(list_clean_df_inj_opz)
df_most_common_inj = pd.DataFrame(c.most_common(), columns = ['薬価基準収載医薬品コード', 'counts'])
df_most_common_inj

#薬価基準収載医薬品コードと薬剤名の対応表を作成する
clean_df_inj_opz_for_correspondence = clean_df_inj_opz.drop(['PT_ID', 'Date', '有＝１'], axis =1)
clean_df_inj_opz_for_correspondence = clean_df_inj_opz_for_correspondence.drop_duplicates(subset = '薬価基準収載医薬品コード')
clean_df_inj_opz_for_correspondence

clean_df_inj_opz_for_correspondence = pd.merge(df_most_common_inj, clean_df_inj_opz_for_correspondence, on = '薬価基準収載医薬品コード')

# 保存
#clean_parsed_df_prescrip_for_correspondence.to_csv('df_most_common_prescrip.csv')
#clean_df_inj_opz_for_correspondence.to_csv('df_most_common_inj.csv')

In [ ]:
#prescriptionで抗がん剤投与がん患者人数
list_clean_parsed_df_prescrip_cleaned = sorted(list(set(list_clean_parsed_df_prescrip)))
pt_id_counts_prescrip = []
for i in list_clean_parsed_df_prescrip_cleaned:
    clean_parsed_df_prescrip_one = clean_parsed_df_prescrip[clean_parsed_df_prescrip['薬価基準収載医薬品コード'] == str(i)]
    pt_id_of_clean_parsed_df_prescrip = sorted(set(list(clean_parsed_df_prescrip_one['PT_ID'])))
    pt_id_counts_one = len(pt_id_of_clean_parsed_df_prescrip)
    pt_id_counts_prescrip.append(pt_id_counts_one)
    
df1 = pd.DataFrame(
    data={'薬価基準収載医薬品コード' : list_clean_parsed_df_prescrip_cleaned, 
          '患者人数': pt_id_counts_prescrip}
          )
df1 = pd.merge(df1, clean_parsed_df_prescrip_for_correspondence, on = '薬価基準収載医薬品コード')
#df1.to_csv('pt_count_prescription.csv')

#injectionで抗がん剤投与がん患者人数
list_clean_df_inj_opz_cleaned = sorted(list(set(list_clean_df_inj_opz)))
pt_id_counts_injection = []
for i in list_clean_df_inj_opz_cleaned:
    clean_df_inj_opz_one = clean_df_inj_opz[clean_df_inj_opz['薬価基準収載医薬品コード'] == str(i)]
    pt_id_of_clean_df_inj_opz = sorted(set(list(clean_df_inj_opz_one['PT_ID'])))
    pt_id_counts_one = len(pt_id_of_clean_df_inj_opz)
    pt_id_counts_injection.append(pt_id_counts_one)
    
df2 = pd.DataFrame(
    data={'薬価基準収載医薬品コード' : list_clean_df_inj_opz_cleaned, 
          '患者人数': pt_id_counts_injection}
          )
df2 = pd.merge(df2, clean_df_inj_opz_for_correspondence, on = '薬価基準収載医薬品コード') 
#df2.to_csv('pt_count_injection.csv')

In [ ]:
#injection Top3
#シスプラチン点滴静注、ランダ注射薬
#4291401
clean_df_inj_opz_4291401 = clean_df_inj_opz[clean_df_inj_opz['薬価基準収載医薬品コード'].str.contains('4291401', na = False)]
pt_id_clean_df_inj_opz_4291401 = sorted(set(list(clean_df_inj_opz_4291401['PT_ID'])))
#カルボプラチン点滴静注、パラプラチン
#4291403
clean_df_inj_opz_4291403 = clean_df_inj_opz[clean_df_inj_opz['薬価基準収載医薬品コード'].str.contains('4291403', na = False)]
pt_id_clean_df_inj_opz_4291403 = sorted(set(list(clean_df_inj_opz_4291403['PT_ID'])))
#タキソール注射液、パクリタキセル注
#4240406
clean_df_inj_opz_4240406 = clean_df_inj_opz[clean_df_inj_opz['薬価基準収載医薬品コード'].str.contains('4240406', na = False)]
pt_id_clean_df_inj_opz_4240406 = sorted(set(list(clean_df_inj_opz_4240406['PT_ID'])))

#perception Top1
#カソデックス錠、ビカルタミド錠　
#4291009
clean_parsed_df_prescrip_4291009 = clean_parsed_df_prescrip[clean_parsed_df_prescrip['薬価基準収載医薬品コード'].str.contains('4291009', na = False)]
pt_id_clean_parsed_df_prescrip_4291009 = sorted(set(list(clean_parsed_df_prescrip_4291009['PT_ID'])))

In [ ]:
#injection おすすめ
#ハーセプチン
#4291406
clean_df_inj_opz_4291406 = clean_df_inj_opz[clean_df_inj_opz['薬価基準収載医薬品コード'].str.contains('4291406', na = False)]
pt_id_clean_df_inj_opz_4291406 = sorted(set(list(clean_df_inj_opz_4291406['PT_ID'])))


#perceptionおすすめ
#グリベック
#4291011
clean_parsed_df_prescrip_4291011 = clean_parsed_df_prescrip[clean_parsed_df_prescrip['薬価基準収載医薬品コード'].str.contains('4291011', na = False)]
pt_id_clean_parsed_df_prescrip_4291011 = sorted(set(list(clean_parsed_df_prescrip_4291011['PT_ID'])))



#肺がんズ

#タグリッソ
#4291045
clean_parsed_df_prescrip_4291045 = clean_parsed_df_prescrip[clean_parsed_df_prescrip['薬価基準収載医薬品コード'].str.contains('4291045', na = False)]
pt_id_clean_parsed_df_prescrip_4291045 = sorted(set(list(clean_parsed_df_prescrip_4291045['PT_ID'])))

#ジオトリフ
#4291030
clean_parsed_df_prescrip_4291030 = clean_parsed_df_prescrip[clean_parsed_df_prescrip['薬価基準収載医薬品コード'].str.contains('4291030', na = False)]
pt_id_clean_parsed_df_prescrip_4291030 = sorted(set(list(clean_parsed_df_prescrip_4291030['PT_ID'])))

#タルセバ
#4291016
clean_parsed_df_prescrip_4291016 = clean_parsed_df_prescrip[clean_parsed_df_prescrip['薬価基準収載医薬品コード'].str.contains('4291016', na = False)]
pt_id_clean_parsed_df_prescrip_4291016 = sorted(set(list(clean_parsed_df_prescrip_4291016['PT_ID'])))

In [ ]:
#pt_id
#保存
pt_id_clean_df_inj_opz_4291401 = pd.DataFrame(pt_id_clean_df_inj_opz_4291401)
pt_id_clean_df_inj_opz_4291401.to_csv('pt_id_clean_df_inj_opz_4291401.csv')

pt_id_clean_df_inj_opz_4291403 = pd.DataFrame(pt_id_clean_df_inj_opz_4291403)
pt_id_clean_df_inj_opz_4291403.to_csv('pt_id_clean_df_inj_opz_4291403.csv')

pt_id_clean_df_inj_opz_4240406 = pd.DataFrame(pt_id_clean_df_inj_opz_4240406)
pt_id_clean_df_inj_opz_4240406.to_csv('pt_id_clean_df_inj_opz_4240406.csv')

pt_id_clean_parsed_df_prescrip_4291009 = pd.DataFrame(pt_id_clean_parsed_df_prescrip_4291009)
pt_id_clean_parsed_df_prescrip_4291009.to_csv('pt_id_clean_parsed_df_prescrip_4291009.csv')
#ビカルタミド

In [ ]:
#pt_id
#保存
pt_id_clean_df_inj_opz_4291406 =pd.DataFrame(pt_id_clean_df_inj_opz_4291406)
pt_id_clean_df_inj_opz_4291406.to_csv('pt_id_clean_df_inj_opz_4291406.csv')

pt_id_clean_parsed_df_prescrip_4291011 = pd.DataFrame(pt_id_clean_parsed_df_prescrip_4291011)
pt_id_clean_parsed_df_prescrip_4291011.to_csv('pt_id_clean_parsed_df_prescrip_4291011.csv')

pt_id_clean_parsed_df_prescrip_4291045 = pd.DataFrame(pt_id_clean_parsed_df_prescrip_4291045)
pt_id_clean_parsed_df_prescrip_4291045.to_csv('pt_id_clean_parsed_df_prescrip_4291045.csv')

pt_id_clean_parsed_df_prescrip_4291030 = pd.DataFrame(pt_id_clean_parsed_df_prescrip_4291030)
pt_id_clean_parsed_df_prescrip_4291030.to_csv('pt_id_clean_parsed_df_prescrip_4291030.csv')

pt_id_clean_parsed_df_prescrip_4291016 = pd.DataFrame(pt_id_clean_parsed_df_prescrip_4291016)
pt_id_clean_parsed_df_prescrip_4291016.to_csv('pt_id_clean_parsed_df_prescrip_4291016.csv')